In [1]:
from train_SVM import load_ml_data

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [2]:
ws = [90, 90]
winL, winR = ws

do_preprocess=True
reduced_DS=False
maxRR=True
use_RR=False
norm_RR=True
compute_morph=['resample_10', 'lbp', 'hbf5', 'wvlt', 'HOS']

In [3]:
tr_features, tr_labels, eval_features, eval_labels = load_ml_data(
    (winL, winR),
    do_preprocess,
    reduced_DS,
    maxRR,
    use_RR,
    norm_RR,
    compute_morph,
    cross_patient=False,
    verbose=True)

  0%|          | 0/44 [00:00<?, ?it/s]

Loading MIT BIH arr (DS12) ...


100%|██████████| 44/44 [08:08<00:00, 11.11s/it]


Saving signal processed data ...
getting rr features ...
working on resample_10 ...
(100693, 14)
time for resample_10: 31.83 sec
working on lbp ...
(100693, 30)
time for lbp: 132.23 sec
working on hbf5 ...
(100693, 45)
time for hbf5: 82.77 sec
working on wvlt ...
(100693, 68)
time for wvlt: 80.94 sec
working on HOS ...
(100693, 78)
time for HOS: 151.54 sec
writing pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS12_rm_bsline_maxRR_norm_RR_resample_10_lbp_hbf5_wvlt_HOS_MLII.pkl...
normalizing the data ... 


In [4]:
print(tr_features.shape)
print(tr_labels.shape)
print(eval_features.shape)
print(eval_labels.shape)

(80554, 78)
(80554,)
(20139, 78)
(20139,)


In [9]:
from train_SVM import calc_class_weights

In [10]:
class_weights = calc_class_weights(tr_labels)

In [11]:
from sklearn import svm

In [12]:
svm_model = svm.SVC(C=0.01, 
                    kernel='rbf', 
                    degree=3, 
                    gamma='auto',
                    coef0=0.0, 
                    shrinking=True, 
                    probability=False, 
                    tol=0.001,
                    cache_size=200, 
                    class_weight=class_weights, 
                    max_iter=-1, 
                    decision_function_shape="ovo", 
                    random_state=None)

In [14]:
from utils import PrintTime

In [15]:
with PrintTime("training the model"):
    svm_model.fit(tr_features, tr_labels)

In [22]:
from sklearn.metrics import f1_score, classification_report

In [18]:
y_pred = svm_model.predict(eval_features)

In [19]:
print("marco f1 score", f1_score(eval_labels, y_pred, average='macro'))

marco f1 score 0.7180920925810662


In [20]:
from constant import AAMI_CLASSES

In [23]:
print(classification_report(eval_labels, y_pred, target_names=AAMI_CLASSES))

              precision    recall  f1-score   support

           F       0.22      0.93      0.35       176
           N       1.00      0.93      0.96     18019
        SVEB       0.50      0.90      0.64       577
         VEB       0.88      0.96      0.92      1367

    accuracy                           0.93     20139
   macro avg       0.65      0.93      0.72     20139
weighted avg       0.97      0.93      0.95     20139



In [25]:
model_svm_path="/home/congyu/dataset/ECG/mitdb/ml_learning/svm_models/baseline_model.joblib.pkl"

In [29]:
import os
import joblib

In [30]:
if model_svm_path:
    # Export model: save/write trained SVM model
    if not os.path.exists(os.path.dirname(model_svm_path)):
        os.makedirs(os.path.dirname(model_svm_path))
    joblib.dump(svm_model, model_svm_path)

In [ ]:
decision_ovo = svm_model.decision_function(eval_features)

In [ ]:
predict_ovo, counter = ovo_voting(decision_ovo, 4)

In [ ]:
perf_measures = compute_AAMI_performance_measures(predict_ovo, labels)
write_AAAMI_results_gamma(output_path, gamma_value, perf_measures, C_value, voting_strategy, DS)